In [1]:
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
import os
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/irtizak/Documents/agenticai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
embeddings.embed_query("hello AI")

[-0.03338826447725296,
 0.03453979268670082,
 0.05947454646229744,
 0.05928615853190422,
 -0.06353536993265152,
 -0.06819585710763931,
 0.08823320269584656,
 0.03444082662463188,
 -0.03278522565960884,
 -0.015814950689673424,
 0.020981717854738235,
 -0.01834026351571083,
 -0.039832208305597305,
 -0.08047077059745789,
 -0.014469246380031109,
 0.033264853060245514,
 0.014259274117648602,
 -0.03404995799064636,
 -0.1429157704114914,
 -0.02308337390422821,
 -0.021380171179771423,
 0.002633556490764022,
 -0.04729270935058594,
 -0.010752714239060879,
 -0.06866799294948578,
 0.031124968081712723,
 0.07594595849514008,
 0.001128278556279838,
 0.011631963774561882,
 -0.03603920713067055,
 0.04483756422996521,
 0.01839076727628708,
 0.12672801315784454,
 -0.001359741436317563,
 0.008206679485738277,
 0.06909968703985214,
 -0.08076363056898117,
 -0.05841312184929848,
 0.05375456064939499,
 0.026227550581097603,
 -0.006828588433563709,
 -0.056358400732278824,
 0.003292981768026948,
 -0.07250183820

In [6]:
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
documents=["what is a capital of USA?",
           "Who is a president of USA?",
           "Who is a prime minister of India?"]

In [8]:
my_query="Narendra modi is prime minister of india?"

In [9]:
document_embedding=embeddings.embed_documents(documents)

In [10]:
document_embedding

[[0.11998695880174637,
  -0.02130262367427349,
  -0.04288090020418167,
  0.06645586341619492,
  -0.0643523707985878,
  -0.04424868896603584,
  0.022408444434404373,
  -0.049873095005750656,
  -0.023437626659870148,
  -0.03397205471992493,
  -0.014048080891370773,
  -0.06065928936004639,
  -0.003906791564077139,
  -0.017782062292099,
  -0.047971028834581375,
  -0.06668157875537872,
  0.004103226587176323,
  -0.01309276930987835,
  0.04439779371023178,
  0.02235068567097187,
  0.0094595430418849,
  -0.020564522594213486,
  -0.00033563413308002055,
  -0.0056857699528336525,
  0.05558694899082184,
  0.025123175233602524,
  -0.00281714741140604,
  0.00875901523977518,
  0.0032552576158195734,
  -0.01596345193684101,
  0.014263682998716831,
  -0.11220849305391312,
  0.08968557417392731,
  -0.031083764508366585,
  -0.02422383800148964,
  0.006152115762233734,
  0.08058717101812363,
  0.018250029534101486,
  0.05568309873342514,
  0.01670265756547451,
  0.015896005555987358,
  0.00034107785904

In [11]:
query_embedding=embeddings.embed_query(my_query)

In [12]:
len(query_embedding)

384

In [13]:
cosine_similarity([query_embedding],document_embedding)

array([[0.11756671, 0.34324563, 0.81413238]])

In [14]:
from sklearn.metrics.pairwise import euclidean_distances

In [15]:
euclidean_distances([query_embedding], document_embedding)

array([[1.32848281, 1.14608407, 0.60970093]])

| Metric            | Similarity Score Range | Behavior                              |
| ----------------- | ---------------------- | ------------------------------------- |
| Cosine Similarity | \[-1, 1]               | Focuses on angle only |
| L2 Distance       | \[0, ∞)                | Focuses on **magnitude + direction**  |


In [16]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [17]:
index=faiss.IndexFlatL2(384)

In [18]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x1316e9470> >

In [19]:
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [20]:
vector_store.add_texts(["AI is future","AI is powerful","Dogs are cute"])

['2467b7a5-b4e5-43a2-988c-61f8be7745f5',
 'c141f13b-7589-4894-b4c0-d3c20aed810d',
 '75bcf951-773c-43d2-a2d7-a9999b2f6466']

In [21]:
vector_store.index_to_docstore_id

{0: '2467b7a5-b4e5-43a2-988c-61f8be7745f5',
 1: 'c141f13b-7589-4894-b4c0-d3c20aed810d',
 2: '75bcf951-773c-43d2-a2d7-a9999b2f6466'}

In [22]:
results = vector_store.similarity_search("Tell me about AI", k=3)

In [23]:
results


[Document(id='c141f13b-7589-4894-b4c0-d3c20aed810d', metadata={}, page_content='AI is powerful'),
 Document(id='2467b7a5-b4e5-43a2-988c-61f8be7745f5', metadata={}, page_content='AI is future'),
 Document(id='75bcf951-773c-43d2-a2d7-a9999b2f6466', metadata={}, page_content='Dogs are cute')]

| Feature               | `Flat`                | `IVF` (Inverted File Index)        | `HNSW` (Graph-based Index)          |
| --------------------- | --------------------- | ---------------------------------- | ----------------------------------- |
| Type of Search     | Exact                 | Approximate (cluster-based)        | Approximate (graph-based traversal) |
| Speed               | Slow (linear scan)    | Fast (search only in top clusters) | Very Fast (graph walk)              |


| Dataset Size              | Recommended Index                 |
| ------------------------- | --------------------------------- |
| UPTO 1L                     | `IndexFlatL2` or `IndexFlatIP`    |
| UPTO 1M                  | `IndexIVFFlat` or `IndexHNSWFlat` |
| > 1M                      | `IndexIVFPQ` or `IndexHNSWFlat`   |


In [24]:
# from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

In [25]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [26]:
vector_store.add_documents(documents=documents)

['dffaf99f-0bed-4158-9878-f43f6dd5ce51',
 '69bea691-60b4-44f1-b62c-45e8b9670397',
 '750e2d8d-2701-471e-bd13-a46edba788f3',
 'c6b8d7cf-e25b-4887-ae83-6dc8c533e8e2',
 'd362f5a8-54f3-4cf0-a8be-2e115e381f94',
 '715056cd-691b-46cb-aa79-82965dbf3a6d',
 'c4b2f2e7-bd2c-42f4-9cdc-2ae636cdf1ce',
 'de926695-84b1-424e-97ec-55419c822b65',
 'ee845171-92e9-489f-9c2d-e887ed927f9a',
 '7011118b-a06e-4ba6-bab2-d1dfd9e855da']

In [27]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2 #hyperparameter
    
)

[Document(id='750e2d8d-2701-471e-bd13-a46edba788f3', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='de926695-84b1-424e-97ec-55419c822b65', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

In [28]:
vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":{"$eq": "tweet"}}
    
)

[Document(id='750e2d8d-2701-471e-bd13-a46edba788f3', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='de926695-84b1-424e-97ec-55419c822b65', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='7011118b-a06e-4ba6-bab2-d1dfd9e855da', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='dffaf99f-0bed-4158-9878-f43f6dd5ce51', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [29]:
result=vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    #k=2 #hyperparameter,
    filter={"source":"news"}
    
)

In [30]:
result[0].metadata

{'source': 'news'}

In [31]:
result[0].page_content

'Robbers broke into the city bank and stole $1 million in cash.'

In [32]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [33]:
retriever.invoke("LangChain provides abstractions to make working with LLMs easy")

[Document(id='750e2d8d-2701-471e-bd13-a46edba788f3', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='de926695-84b1-424e-97ec-55419c822b65', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='7011118b-a06e-4ba6-bab2-d1dfd9e855da', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :(')]

In [34]:
# inmemory(server)
# ondisk(server)
# cloud(yet to discuss)

In [35]:
vector_store.save_local("today's class faiss index")

In [36]:
new_vector_store=FAISS.load_local(
  "today's class faiss index",embeddings ,allow_dangerous_deserialization=True
)

In [37]:
new_vector_store.similarity_search("langchain")

[Document(id='750e2d8d-2701-471e-bd13-a46edba788f3', metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(id='de926695-84b1-424e-97ec-55419c822b65', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!'),
 Document(id='d362f5a8-54f3-4cf0-a8be-2e115e381f94', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='c4b2f2e7-bd2c-42f4-9cdc-2ae636cdf1ce', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [38]:
from langchain_community.document_loaders import PyPDFLoader

In [39]:
os.getcwd()

'/Users/irtizak/Documents/agenticai/2-Langchain Basics/2.4-VectorDatabase/FAISS'

In [40]:
loader = PyPDFLoader("llama2.pdf")

In [41]:
len(loader.load())

77

In [42]:
pages=loader.load()

In [43]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [45]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)

In [46]:
split_docs = splitter.split_documents(pages)

In [47]:
len(split_docs)

615

In [48]:
index=faiss.IndexFlatIP(384)
vector_store=FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [49]:
vector_store.add_documents(documents=split_docs)

['761f2b72-3adb-4af0-9f2b-f69a5d56caa1',
 'dd6341fa-4e56-4e06-a257-81eebc856f10',
 'd351a046-c85b-4901-9933-a1afaa265eab',
 'd4bc307a-d418-4d34-a4d6-7ef81361eb4c',
 'fd2097c7-dfb4-4189-9211-f2fe65a111f1',
 'ba2ded26-aa80-45cb-b03a-b55759bfe275',
 '06fce4a4-28fe-486e-b38b-31f0ca64d5ee',
 'f93d8983-a2a0-4401-aec6-683a9bd7f4ca',
 'a531f22c-ca73-44d3-a9ff-709ebab73e3b',
 'b474bb8d-7340-4070-b678-8b764aeec5be',
 'a4f5e612-778c-47a5-93ef-6db7b2fac751',
 'c75272cc-7291-46cb-a6d6-53d03db0be00',
 'da217f70-7ada-4e76-b495-7535edc7db8a',
 'eeee7764-aa12-4933-b891-d42044454ab7',
 '321015b4-648e-41cc-aab9-d5706792bd74',
 'f02a7989-734e-4675-be58-f5910f83db61',
 'b561efd0-50bf-4fa4-89e6-e2d765e72143',
 '9d2611fd-9b97-4bce-a6da-3360609da536',
 'fe113a72-1193-4572-9728-395839c156d9',
 '64a904a0-b1e6-494b-9945-cb1320fce5e9',
 '8c00d4e4-0c8d-4c29-99f3-f5687ed04faf',
 'c7c0f4d5-95cb-45d1-b603-2b34b24623f0',
 '5d4957f3-091e-4370-afb6-0fa7ba37bccb',
 '18faa38c-bec6-40e6-9786-e4ddbc171fa8',
 '8a009c8a-2ced-

In [50]:
retriever=vector_store.as_retriever(
    search_kwargs={"k": 10} #hyperparameter
)

In [51]:
retriever.invoke("what is llama model?")

[Document(id='8a009c8a-2ced-436e-8f98-15fce5c4577e', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'llama2.pdf', 'total_pages': 77, 'page': 3, 'page_label': '4'}, page_content='work (Section 6), and conclusions (Section 7).\n‡https://ai.meta.com/resources/models-and-libraries/llama/\n§We are delaying the release of the 34B model due to a lack of time to sufficiently red team.\n¶https://ai.meta.com/llama\n‖https://github.com/facebookresearch/llama\n4'),
 Document(id='c8ea933c-a622-48d6-b633-8800b5c16da7', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:3

In [52]:
from langchain_google_genai import ChatGoogleGenerativeAI
model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [53]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [54]:
import pprint

In [55]:
pprint.pprint(prompt.messages)

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]

In [56]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [57]:
# context(retriever),prompt(hub),model(google),parser(langchain)

In [58]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [59]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [60]:
rag_chain.invoke("what is llama model?")

'Llama is a large language model developed by Meta.  It has various versions, including tuned versions for assistant-like chat and pretrained models adaptable to different natural language tasks.  A Stanford version, Alpaca, is also instruction-following.'